## Import Libraries

In [4]:
import cv2
import numpy as np
import mediapipe as mp
from scipy.signal import butter, filtfilt
from mediapipe.python.solutions.pose import POSE_CONNECTIONS

## Butterworth filter functions

Here we use **Butterworth filters** via Scipy library. [signal.butter](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.butter.html) is a function provided by the [scipy.signal](https://docs.scipy.org/doc/scipy/reference/signal.html) module in Python. It is used to design digital Butterworth filters, which are a type of infinite impulse response (IIR) filter commonly used for signal processing tasks, including filtering EEG signals. The Butterworth filter is characterized by having a flat frequency response in its passband and a gradual roll-off in its stopband.

In [1]:
def butter_lowpass(cutoff, fs, order=2):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq   # normalized cutoff frequency
    b, a = butter(order, normal_cutoff, btype = "low", analog = False)
    return b, a


def butter_lowpass_filter(data, cutoff, fs, order=2):
    b, a = butter_lowpass(cutoff, fs, order=order)
    return filtfilt(b, a, data)
    

## Kalman filter function
Here we use a simple 1D **Kalman Filter** implemented in Python. The KalmanFilter1D class models the process and measurement uncertainties and continuously updates its estimate as new data arrives. The Kalman Filter is an optimal recursive algorithm that combines prior predictions with new noisy measurements to produce a more accurate estimate of the true value. It is widely used in signal processing, control systems, and sensor fusion due to its ability to reduce noise and track system states over time.

### Kalman Filter 1D - Key Formulas

#### 1. Prior (Prediction) Estimate
The prior (predicted) estimate is the prediction of the current state before seeing the measurement:

$$
x_{\text{prior}} = x_{\text{posteri}}
$$

#### 2. Prior Error Estimate
The uncertainty of the prior estimate increases due to process noise:

$$
P_{\text{prior}} = P_{\text{posteri}} + Q
$$

Where:  
- P_prior is the predicted (prior) error estimate  
- P_posteri is the previous posterior error estimate  
- Q is the process noise variance

#### 3. Kalman Gain
The Kalman Gain determines how much we trust the measurement vs. the prediction:

$$
K = \frac{P_{\text{prior}}}{P_{\text{prior}} + R}
$$

Where:  
- K is the Kalman Gain  
- R is the measurement noise variance

#### 4. Posterior (Updated) Estimate
Update the estimate with the new measurement:

$$
x_{\text{posteri}} = x_{\text{prior}} + K \cdot (z - x_{\text{prior}})
$$

Where:  
- x_posteri is the updated (posterior) estimate  
- z is the new measurement

#### 5. Posterior Error Estimate
Update the error covariance after measurement:

$$
P_{\text{posteri}} = (1 - K) \cdot P_{\text{prior}}
$$

Where:  
- P_posteri is the updated error estimate



In [1]:
class KalmanFilter1D:
    def __init__(self, process_variance=0.001, measurement_variance=0.01):
        self.process_variance = process_variance
        self.measurement_variance = measurement_variance
        self.posteri_estimate = 0.0
        self.posteri_error_estimate = 1.0

    def update(self, measurement):
        priori_estimate = self.posteri_estimate
        priori_error_estimate = self.posteri_error_estimate + self.process_variance
        blending_factor = priori_error_estimate / (priori_error_estimate + self.measurement_variance)
        self.posteri_estimate = priori_estimate + blending_factor * (measurement - priori_estimate)
        self.posteri_error_estimate = (1 - blending_factor) * priori_error_estimate
        return self.posteri_estimate

## 4. Load MediaPipe Pose Model
Initialize the MediaPipe Pose model for landmark detection.

In [ ]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

cap = cv2.VideoCapture("pose.mp4")

fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0 or np.isnan(fps):
    fps = 30

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter("pose-result.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

In [ ]:
# Kalman filters

kalman_filters_x = [KalmanFilter1D() for i in range(33)]
kalman_filters_y = [KalmanFilter1D() for i in range(33)]

frames_data = []
last_valid_coords = [(0, 0)] * 33    # keep last valid coordinates for missing landmarks
# collect coordinates
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        coords = [(lm.x, lm.y) for lm in landmarks]
        last_valid_coords = coords
    else:
        coords = last_valid_coords

    frames_data.append(coords)

cap.release()

## Apply Butterworth Filtering
Filter the landmark coordinate sequences to remove noise.

In [ ]:
# Apply Butterworth
all_x = [[] for i in range(33)]
all_y = [[] for i in range(33)]

for i in range(33):
    x_series = [f[i][0] for f in frames_data]
    y_series = [f[i][1] for f in frames_data]

    if len(x_series) > 9:
        x_filtered = butter_lowpass_filter(x_series, cutoff=3, fs=fps)
        y_filtered = butter_lowpass_filter(y_series, cutoff=3, fs=fps)
    else:
        x_filtered = x_series
        y_filtered = y_series

    all_x[i] = x_filtered
    all_y[i] = y_filtered

## Kalman Smoothing and Drawing
Apply Kalman filter to the already low-pass filtered coordinates, draw points and connections on each frame, and save the output.

In [ ]:
# draw with Kalman smoothing
cap = cv2.VideoCapture("pose.mp4")
frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    for i in range(33):
        kx = kalman_filters_x[i].update(all_x[i][frame_idx])
        ky = kalman_filters_y[i].update(all_y[i][frame_idx])

        cx = int(kx * width)
        cy = int(ky * height)
        cv2.circle(frame, (cx, cy), 4, (0, 255, 0), -1)
    for connection in POSE_CONNECTIONS:
        start_idx, end_idx = connection
        start_x = int(all_x[start_idx][frame_idx] * width)
        start_y = int(all_y[start_idx][frame_idx] * height)
        end_x = int(all_x[end_idx][frame_idx] * width)
        end_y = int(all_y[end_idx][frame_idx] * height)
        cv2.line(frame, (start_x, start_y), (end_x, end_y), (0, 255, 255), 2)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()
cv2.destroyAllWindows()